In [2]:
import pandas as pd
import psycopg2
import geopandas as gpd
from shapely import wkt

In [26]:
conn = None
try:
    conn_string = "dbname='vacantlotdb' user='postgres' host='localhost' password='password'"
    conn = psycopg2.connect(conn_string)
    print("Connecting to database\n->{}".format(conn_string))

    # Create a cursor and execute the query
    # Assuming the geometry column in your table is named 'geometry'
    query = """SELECT *, ST_AsText(geometry) AS geom_wkt FROM public."vacant_properties_end";"""
    
    cur = conn.cursor()
    cur.execute(query)
    
    # Fetch all the results
    records = cur.fetchall()
    
    # Get the column names from the cursor description
    columns = [desc[0] for desc in cur.description]

    # Convert to a GeoDataFrame
    # Assuming that the geometry is returned as the last column in the cursor
    gdf = gpd.GeoDataFrame(records, columns=columns)
    
    # Convert WKT geometry to shapely geometries
    gdf['geometry'] = gdf['geom_wkt'].apply(wkt.loads)
    gdf = gdf.drop('geom_wkt', axis=1)
    gdf.set_geometry('geometry', inplace=True)
    
    print(gdf.head())

except Exception as e:
    print("An error occurred: {}".format(e))

finally:
    if conn is not None:
        conn.close()


Connecting to database
->dbname='vacantlotdb' user='postgres' host='localhost' password='password'
   OBJECTID_left             ADDRESS                              OWNER1  \
0           3667  3870 WYALUSING AVE                    BRYANT SHIRLEY R   
1           3668      1542 W YORK ST              PHILLY HOLDINGS 18 LLC   
2           3668    439 HANSBERRY ST                MOSER MICHELLE MARIE   
3           3669     1837 W ERIE AVE          STRATEGIC EQUITY GROUP INC   
4           3669     5123 MALCOLM ST  CUMBERLAND PROPERTY ASSOCIATES LLC   

  OWNER2               BLDG_DESC     OPA_ID LNIADDRESSKEY  \
0   None       ROW 2 STY MASONRY  243180600        625621   
1   None     VAC LAND RES < ACRE  161275000        629200   
2   None  SEMI/DET 3 STY MASONRY  123107600        332149   
3   None     VAC LAND RES < ACRE  131063100        139111   
4   None       ROW 2 STY MASONRY  511094500        406914   

  COUNCILDISTRICT_left ZONINGBASEDISTRICT ZIPCODE  ...  \
0                  

In [28]:
gdf.head()

,OBJECTID_left,ADDRESS,OWNER1,OWNER2,BLDG_DESC,OPA_ID,LNIADDRESSKEY,COUNCILDISTRICT_left,ZONINGBASEDISTRICT,ZIPCODE,...,updated_at,guncrime_density,opa_number,total_due,is_actionable,payment_agreement,num_years_owed,most_recent_year_owed,total_assessment,sheriff_sale
0,3667,3870 WYALUSING AVE,BRYANT SHIRLEY R,None,ROW 2 STY MASONRY,243180600,625621,3,RSA-5,19104,...,2013-03-19 13:41:50.743000-04:00,Bottom 50%,243180600.0,984.86,true,false,2.0,2018.0,42000.0,N
1,3668,1542 W YORK ST,PHILLY HOLDINGS 18 LLC,None,VAC LAND RES < ACRE,161275000,629200,5,RM-1,19132,...,2013-03-19 13:41:50.743000-04:00,Top 1%,NaN,NaN,None,None,NaN,NaN,NaN,None
2,3668,439 HANSBERRY ST,MOSER MICHELLE MARIE,None,SEMI/DET 3 STY MASONRY,123107600,332149,8,RSA-3,19144,...,2013-03-19 13:41:50.743000-04:00,Bottom 50%,123107600.0,9711.47,true,false,2.0,2021.0,244900.0,N
3,3669,1837 W ERIE AVE,STRATEGIC EQUITY GROUP INC,None,VAC LAND RES < ACRE,131063100,139111,8,RM-1,19140,...,2013-03-19 13:41:50.743000-04:00,Top 50%,NaN,NaN,None,None,NaN,NaN,NaN,None
4,3669,5123 MALCOLM ST,CUMBERLAND PROPERTY ASSOCIATES LLC,None,ROW 2 STY MASONRY,511094500,406914,3,RM-1,19143,...,2013-03-19 13:41:50.743000-04:00,Bottom 50%,NaN,NaN,None,None,NaN,NaN,NaN,None


In [ ]:
### needs:
# property ownership (city, public but not city, or individual?)--this comes from 'agency' column in city_owned_properties. If agency == PLB, go to land bank. otherwise, can't do anything.
# sale date (last six months?) # complete
# facing foreclosure?
# vacant, unsafe, or notorious? (from 311 data)
# tax delinquent? (boolean) # complete
# payment plan? (boolean) # complete
# land or building? 9boolean)
# cost (> 1000) # complete

In [29]:
gdf.columns

Index(['OBJECTID_left', 'ADDRESS', 'OWNER1', 'OWNER2', 'BLDG_DESC', 'OPA_ID',
       'LNIADDRESSKEY', 'COUNCILDISTRICT_left', 'ZONINGBASEDISTRICT',
       'ZIPCODE', 'LAND_RANK', 'Shape__Area_left', 'Shape__Length_left',
       'BUILD_RANK', 'OBJECTID_right', 'PIN', 'MAPREG_1', 'AGENCY', 'OPABRT',
       'LOCATION', 'STATUS_1', 'COUNCILDISTRICT_right', 'SIDEYARDELIGIBLE',
       'ZONING', 'Shape__Area_right', 'Shape__Length_right', 'COMM_PARTN',
       'li_code_violations', 'li_complaints', 'rco_info', 'geometry',
       'tree_canopy_gap', 'name', 'listname', 'neighborhood', 'shape_leng',
       'shape_area', 'cartodb_id', 'created_at', 'updated_at',
       'guncrime_density', 'opa_number', 'total_due', 'is_actionable',
       'payment_agreement', 'num_years_owed', 'most_recent_year_owed',
       'total_assessment', 'sheriff_sale'],
      dtype='object')

In [36]:
keep_cols = ['ADDRESS', 'OWNER1', 'OWNER2', 'BLDG_DESC', 'OPA_ID',
       'MAPREG_1', 'AGENCY', 'OPABRT',
       'LOCATION','COMM_PARTN',
       'li_code_violations', 'li_complaints', 'geometry',
       'tree_canopy_gap', 'listname',
       'guncrime_density', 'total_due', 'is_actionable',
       'payment_agreement', 'num_years_owed', 'most_recent_year_owed',
       'total_assessment', 'sheriff_sale']

In [37]:
filt_gdf = gdf[keep_cols]

In [38]:
filt_gdf.head()

,ADDRESS,OWNER1,OWNER2,BLDG_DESC,OPA_ID,MAPREG_1,AGENCY,OPABRT,LOCATION,COMM_PARTN,...,tree_canopy_gap,listname,guncrime_density,total_due,is_actionable,payment_agreement,num_years_owed,most_recent_year_owed,total_assessment,sheriff_sale
0,3870 WYALUSING AVE,BRYANT SHIRLEY R,None,ROW 2 STY MASONRY,243180600,None,None,None,None,None,...,0.110429,East Parkside,Bottom 50%,984.86,true,false,2.0,2018.0,42000.0,N
1,1542 W YORK ST,PHILLY HOLDINGS 18 LLC,None,VAC LAND RES < ACRE,161275000,None,None,None,None,None,...,0.165151,Stanton,Top 1%,NaN,None,None,NaN,NaN,NaN,None
2,439 HANSBERRY ST,MOSER MICHELLE MARIE,None,SEMI/DET 3 STY MASONRY,123107600,None,None,None,None,None,...,0.038058,"Germantown, Westside",Bottom 50%,9711.47,true,false,2.0,2021.0,244900.0,N
3,1837 W ERIE AVE,STRATEGIC EQUITY GROUP INC,None,VAC LAND RES < ACRE,131063100,None,None,None,None,None,...,0.085772,Tioga,Top 50%,NaN,None,None,NaN,NaN,NaN,None
4,5123 MALCOLM ST,CUMBERLAND PROPERTY ASSOCIATES LLC,None,ROW 2 STY MASONRY,511094500,None,None,None,None,None,...,0.275427,Cedar Park,Bottom 50%,NaN,None,None,NaN,NaN,NaN,None


In [ ]:
cond1 = joined_gdf["public_owner"]
val1 = "Deed from City"

cond2 = (~joined_gdf["public_owner"]) & (joined_gdf["opa_number"].isna())
val2 = "Work with owner"

cond3 = (~joined_gdf["public_owner"]) & (joined_gdf["opa_number"].notna()) & (joined_gdf["num_years_owed"] > 3)
val3 = "Sherrif's sale"

cond4 = (~joined_gdf["public_owner"]) & (joined_gdf["opa_number"].notna()) & (joined_gdf["num_years_owed"] <= 3) & (joined_gdf["owner"].str.contains("LLC"))
val4 = "Force foreclosure"

cond5 = (~joined_gdf["public_owner"]) & (joined_gdf["opa_number"].notna()) & (joined_gdf["num_years_owed"] <= 3) & (~joined_gdf["owner"].str.contains("LLC")) & (joined_gdf["market_value"] < 100000)
val5 = "Buy from owner"

cond6 = (~joined_gdf["public_owner"]) & (joined_gdf["opa_number"].notna()) & (joined_gdf["num_years_owed"] <= 3) & (~joined_gdf["owner"].str.contains("LLC")) & (joined_gdf["market_value"] >= 100000)
val6 = "Turn over to developer"

# Use np.select to create the new column based on the conditions and values
joined_gdf["acquisition_process"] = np.select([cond1, cond2, cond3, cond4, cond5, cond6], [val1, val2, val3, val4, val5, val6], default="Too complicated")

In [39]:
conn = None
try:
    conn_string = "dbname='vacantlotdb' user='postgres' host='localhost' password='password'"
    conn = psycopg2.connect(conn_string)
    print("Connecting to database\n->{}".format(conn_string))

    # Create a cursor and execute the query
    # Assuming the geometry column in your table is named 'geometry'
    query = """SELECT *, ST_AsText(geometry) AS geom_wkt FROM public."city_owned_properties";"""
    
    cur = conn.cursor()
    cur.execute(query)
    
    # Fetch all the results
    records = cur.fetchall()
    
    # Get the column names from the cursor description
    columns = [desc[0] for desc in cur.description]

    # Convert to a GeoDataFrame
    # Assuming that the geometry is returned as the last column in the cursor
    city_props_gdf = gpd.GeoDataFrame(records, columns=columns)
    
    # Convert WKT geometry to shapely geometries
    city_props_gdf['geometry'] = city_props_gdf['geom_wkt'].apply(wkt.loads)
    city_props_gdf = city_props_gdf.drop('geom_wkt', axis=1)
    city_props_gdf.set_geometry('geometry', inplace=True)
    
    print(city_props_gdf.head())

except Exception as e:
    print("An error occurred: {}".format(e))

finally:
    if conn is not None:
        conn.close()

Connecting to database
->dbname='vacantlotdb' user='postgres' host='localhost' password='password'
                                            geometry  OBJECTID           PIN  \
0  POLYGON ((2699025.903 247583.124, 2699023.347 ...         1  1.001191e+09   
1  POLYGON ((2668978.552 241791.255, 2668976.670 ...         2  1.001168e+09   
2  POLYGON ((2697211.780 244521.343, 2697210.142 ...         3  1.001170e+09   
3  POLYGON ((2680464.106 243522.484, 2680448.457 ...         4  1.001243e+09   
4  POLYGON ((2703401.044 251276.499, 2703402.818 ...         5  1.001306e+09   

     MAPREG_1 AGENCY     OPABRT           LOCATION  \
0  028N200087    PLB  191385801      518 Edgley St   
1  064N040095    PRA  343128700   412 N Daggett St   
2  012N040184    PLB  202130501   1512 N Darien St   
3  098N020085    PLB  062179200  4155 W Girard Ave   
4  035N060061    PLB  071387010        2841 Kip St   

                                      STATUS_1  COUNCILDISTRICT  \
0                           

In [40]:
city_props_gdf.head()

,geometry,OBJECTID,PIN,MAPREG_1,AGENCY,OPABRT,LOCATION,STATUS_1,COUNCILDISTRICT,SIDEYARDELIGIBLE,ZONING,Shape__Area,Shape__Length
0,"POLYGON ((2699025.903 247583.124, 2699023.347 ...",1,1.001191e+09,028N200087,PLB,191385801,518 Edgley St,Owned - Available,7,Yes,RSA-5,440.999207,97.851306
1,"POLYGON ((2668978.552 241791.255, 2668976.670 ...",2,1.001168e+09,064N040095,PRA,343128700,412 N Daggett St,Owned - On Hold,4,No,RM-1,1515.301208,230.369008
2,"POLYGON ((2697211.780 244521.343, 2697210.142 ...",3,1.001170e+09,012N040184,PLB,202130501,1512 N Darien St,"Owned - Processing Applicant, Not Available",5,Yes,RSA-5,568.292297,117.110154
3,"POLYGON ((2680464.106 243522.484, 2680448.457 ...",4,1.001243e+09,098N020085,PLB,062179200,4155 W Girard Ave,Owned - On Hold,3,No,RM-1,1858.613586,263.961210
4,"POLYGON ((2703401.044 251276.499, 2703402.818 ...",5,1.001306e+09,035N060061,PLB,071387010,2841 Kip St,Owned - Available,7,Yes,RSA-5,647.454773,129.331994
